# Imports libraries

In [1]:
import pandas as pd
import numpy as np 
import datetime
import time
import sys
from scipy import stats
from statsmodels.stats import stattools
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import multiprocessing as mp

pd.options.plotting.backend = "plotly"
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Include py file

In [2]:
from Differentiation import *

# Import Tick data

In [3]:
dir = "https://raw.githubusercontent.com/risk-labratory/data/main/"
url = dir + "IVE_2020.csv"
df = pd.read_csv(url, header = 0)
df['dates'] = pd.to_datetime(df['dates'])
df.set_index('dates', inplace = True, drop = True)
df.drop_duplicates(inplace = True)
df = df[(df.index.hour >= 9) & (df.index.hour < 16)]
df.head()

,price,bid,ask,size
dates,,,,
2020-01-02 09:30:00,130.68,130.59,130.60,20625
2020-01-02 09:30:01,130.50,130.50,130.77,200
2020-01-02 09:30:04,130.53,130.52,130.78,100
2020-01-02 09:30:04,130.55,130.52,130.78,100
2020-01-02 09:30:04,130.53,130.52,130.78,200


In [4]:
ohlcv = timeBar(df, frequency = "1B")
ohlcv.dropna(inplace = True)
close = ohlcv.close
ohlcv.head()

,open,high,low,close,volume,ValueOfTrades,PriceMean,TickCount,PriceMeanLogReturn
dates,,,,,,,,,
2020-01-03,129.33,129.9874,129.2912,129.6400,526340,129.751774,129.719157,922,-0.003845
2020-01-06,129.00,129.8952,128.9300,129.8952,655431,129.548003,129.493223,770,-0.001743
2020-01-07,129.52,129.5800,129.1405,129.3800,413423,129.376731,129.357347,908,-0.001050
2020-01-08,129.38,130.2999,129.2400,129.7600,449383,129.881903,129.858126,1028,0.003864
2020-01-09,130.30,130.3800,129.9200,130.3168,376142,130.161216,130.161563,614,0.002334


# Snippet 5.1

In [5]:
plotWeights([0, 1], 11, 6)

In [6]:
plotWeights([1, 2], 11, 6)

# Snippet 5.2

In [7]:
fracDiff(pd.DataFrame(close), degree = 0.4, threshold = 0.01)

,close
2020-07-29,12.300457
2020-07-30,10.545606
2020-07-31,10.529309
2020-08-03,10.973636
2020-08-04,11.471090
...,...
2020-12-23,10.861985
2020-12-24,10.942959
2020-12-29,11.069820
2020-12-30,11.423306


In [8]:
close_fd = fracDiff(pd.DataFrame(close), degree = 0.4, threshold = 1).iloc[:, 0]
close_fd_c = fracDiff(pd.DataFrame(close), degree = 0.4, threshold = 0.01).iloc[:, 0]
fig = make_subplots(specs = [[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = close.index, y = close, name = "close"), secondary_y = False)
fig.add_trace(go.Scatter(x = close_fd.index, y = close_fd, name = "Diff"), secondary_y = True)
fig.add_trace(go.Scatter(x = close_fd_c.index, y = close_fd_c, name = "Diff Controlled"), secondary_y = True)
fig.update_yaxes(title_text = "Price", secondary_y = False)
fig.update_yaxes(title_text = "Diff Ret", secondary_y = True)
fig.show()

# Snippet 5.3

In [9]:
fracDiffFixed(pd.DataFrame(close), degree = 0.3, threshold = 1e-3)

,close
2020-04-09,25.786355
2020-04-14,24.827002
2020-04-15,21.274337
2020-04-16,21.721812
2020-04-17,25.589026
...,...
2020-12-23,28.446815
2020-12-24,28.608569
2020-12-29,28.783656
2020-12-30,29.180724


In [10]:
close_ffd = fracDiffFixed(pd.DataFrame(close), degree = 0.3, threshold = 1e-3).iloc[:, 0]
fig = make_subplots(specs = [[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x = close.index, y = close, name = "close"), secondary_y = False)
fig.add_trace(go.Scatter(x = close_ffd.index, y = close_ffd, name = "FFD"), secondary_y = True)
fig.update_yaxes(title_text = "Price", secondary_y = False)
fig.update_yaxes(title_text = "Diff Ret", secondary_y = True)
fig.show()

# Snippet 5.4

In [11]:
out = minFFD(ohlcv)
out.head()

,adfStat,pVal,lags,nObs,95% conf,corr
0.0,-1.569758,0.498765,1.0,242.0,-2.873559,1.000000
0.1,-1.859458,0.351367,1.0,237.0,-2.873814,0.996445
0.2,-2.364270,0.152076,1.0,236.0,-2.873866,0.974398
0.3,-3.046800,0.030749,1.0,235.0,-2.873919,0.920690
0.4,-3.846095,0.002469,1.0,236.0,-2.873866,0.840757


In [12]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=out.index, y=out['corr'], name="corr"), secondary_y=False)
fig.add_trace(go.Scatter(x=out.index, y=out['adfStat'], name="adfStat"), secondary_y=True)
fig.add_hline(y=out['95% conf'].mean(), line_width=3, line_dash="dash", line_color="green", secondary_y=True)
fig.update_yaxes(title_text="corr", secondary_y=False)
fig.update_yaxes(title_text="adfStat", secondary_y=True)
fig.show()